# Calculate the mean inside eddies from satellite data
Code written by: Billy Atkinson (watkinson@umces.edu) \
Paper Citation: Mason, E., A. Pascual, and J. C. McWilliams, 2014: A new sea surface height–based code for oceanic mesoscale eddy tracking. J. Atmos. Oceanic Technol., 31, 1181–1188, doi:10.1175/JTECH-D-14-00019.1. \
Source Code: __[Mean Inside Eddies](https://py-eddy-tracker.readthedocs.io/en/latest/python_module/02_eddy_identification/pet_interp_grid_on_dataset.html#sphx-glr-python-module-02-eddy-identification-pet-interp-grid-on-dataset-py)__ \
Date of last revision: 10/20/2022 

In [1]:
import xarray as xr 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import netCDF4 as nc
import datetime as dt
import matplotlib.cm as cm
from mpl_toolkits.basemap import Basemap
from scipy.stats import linregress
import numpy.ma as ma
import glob
import os
from py_eddy_tracker.dataset.grid import RegularGridDataset
from py_eddy_tracker.observations.observation import EddiesObservations

In [5]:
def grid_stat(x_c, y_c, grid, x, y, result, circular=False, method="mean"):
    """
    Compute the mean or the max of the grid for each contour

    :param array_like x_c: the grid longitude coordinates
    :param array_like y_c: the grid latitude coordinates
    :param array_like grid: grid value
    :param array_like x: longitude of contours
    :param array_like y: latitude of contours
    :param array_like result: return values
    :param bool circular: True if grid is wrappable
    :param str method: 'mean', 'max'
    """
    # FIXME : how does it work on grid bound
    nb = result.shape[0]
    xstep, ystep = x_c[1] - x_c[0], y_c[1] - y_c[0]
    x0, y0 = x_c - xstep / 2.0, y_c - ystep / 2.0
    nb_x = x_c.shape[0]
    max_method = "max" == method
    mean_method = "mean" == method
    count_method = "count" == method
    for elt in range(nb):
        v = create_vertice(x[elt], y[elt])
        (x_start, x_stop), (y_start, y_stop) = bbox_indice_regular(
            v, x0, y0, xstep, ystep, 1, circular, nb_x
        )
        i, j = get_pixel_in_regular(v, x_c, y_c, x_start, x_stop, y_start, y_stop)

        if count_method:
            result[elt] = i.shape[0]
        elif mean_method:
            v_sum = 0
            for i_, j_ in zip(i, j):
                v_sum += np.nansum(grid[i_, j_])
            nb_ = i.shape[0]
            # FIXME : how does it work on grid bound,
            if nb_ == 0:
                result[elt] = nan
            else:
                result[elt] = v_sum / nb_
        elif max_method:
            v_max = -1e40
            for i_, j_ in zip(i, j):
                v_max = max(v_max, grid[i_, j_])
            result[elt] = v_max

In [6]:
def interp_grid(
        self, grid_object, varname, method="center", dtype=None, intern=None
    ):
        """
        Interpolate a grid on a center or contour with mean, min or max method

        :param grid_object: Handler of grid to interp
        :type grid_object: py_eddy_tracker.dataset.grid.RegularGridDataset
        :param str varname: Name of variable to use
        :param str method: 'center', 'mean', 'max', 'min', 'nearest'
        :param str dtype: if None we use var dtype
        :param bool intern: Use extern or intern contour

        .. minigallery:: py_eddy_tracker.EddiesObservations.interp_grid
        """
        if method in ("center", "nearest"):
            return grid_object.interp(varname, self.longitude, self.latitude, method)
        elif method in ("min", "max", "mean", "count"):
            x0 = grid_object.x_bounds[0]
            x_name, y_name = self.intern(False if intern is None else intern)
            x_ref = ((self.longitude - x0) % 360 + x0 - 180).reshape(-1, 1)
            x, y = (self[x_name] - x_ref) % 360 + x_ref, self[y_name]
            grid = grid_object.grid(varname)
            result = empty(self.shape, dtype=grid.dtype if dtype is None else dtype)
            min_method = method == "min"
            grid_stat(
                grid_object.x_c,
                grid_object.y_c,
                -grid if min_method else grid,
                x,
                y,
                result,
                grid_object.is_circular(),
                method="max" if min_method else method,
            )
            return -result if min_method else result
        else:
            raise Exception(f'method "{method}" unknown')

In [48]:
a = EddiesObservations.load_file('/data/watkinson/Fall2022/Eddy_data/GoM_tuned_eddies/Anticyclonic/Anticyclonic_20030101.nc')#anticyclonic eddy data
#c = EddiesObservations.load_file(cycl_files[n])#cyclonic eddy data
sat_data = RegularGridDataset(('/data/watkinson/Fall2022/Sat_data/1day_regrid/CAFE_GOM_day_20030101.nc'),"lon","lat",nan_masking=True)
sat_data
grid = sat_data.grid("chlor_a")
print(np.shape(grid[0]))
result = np.empty(a.shape, dtype=grid.dtype)# if dtype is None else dtype)
print(result)
nb = result.shape[0]
print(nb)
for elt in range(nb):
    result[elt] = i.shape[0]
print(results)

We assume pixel position of grid is centered for /data/watkinson/Fall2022/Sat_data/1day_regrid/CAFE_GOM_day_20030101.nc


(346,)
[1.51e-43 1.36e-43 1.63e-43 1.51e-43 1.56e-43 1.54e-43]
6


In [68]:

################take satelite files pats, sort them and load them into list
sat_files = sorted(glob.glob('/data/watkinson/Fall2022/Sat_data/1day_regrid/*.nc'))
#SSH_files = sorted(glob.glob('/data/watkinson/Fall2022/SSH_data/Cropped_SSH_Data/*.nc'))

sat_files = sat_files[0:3]
#print(SSH_files.index('/data/watkinson/Fall2022/SSH_data/Cropped_SSH_Data/hycom_gomu_501_20030101_cropped.nc'))
#SSH_files = SSH_files[181:]
#print(len(sat_files))
#print(sat_files)

################take anticyclonic eddy files paths, sort them and load them into list
#anti_files = sorted(glob.glob('/data/watkinson/Fall2022/Eddy_data/Original_eddies/Anticyclonic/Anticyclonic_*.nc'))
#anti_files = anti_files[508:1965]

anti_files = sorted(glob.glob('/data/watkinson/Fall2022/Eddy_data/GoM_tuned_eddies/Anticyclonic/Anticyclonic_*.nc'))

print(len(anti_files))
#take cyclonic eddy files paths, sort them and load them into list
#cycl_files = sorted(glob.glob('/data/watkinson/Fall2022/Eddy_data/Original_eddies/Cyclonic/Cyclonic_*.nc'))
#cycl_files = cycl_files[508:1965]
cycl_files = sorted(glob.glob('/data/watkinson/Fall2022/Eddy_data/GoM_tuned_eddies/Cyclonic/Cyclonic_*.nc'))
print(len(cycl_files))
'''
#take satelite files pats, sort them and load them into list
#sat_files = sorted(glob.glob('/data/watkinson/Summer2022/data/cafe/1day_regrid/*.nc'))

#print(len(sat_files))
#take anticyclonic eddy files paths, sort them and load them into list
#anti_files = sorted(glob.glob('/data/watkinson/Summer2022/data/eddies/Anticyclonic/Anticyclonic_2004*.nc'))
#anti_files = sorted(glob.glob('/data/watkinson/Fall2022/Eddy_data/GoM_tuned_eddies/Anticyclonic/Anticyclonic_2004*.nc'))
#print(len(anti_files))
#take cyclonic eddy files paths, sort them and load them into list
#cycl_files = sorted(glob.glob('/data/watkinson/Summer2022/data/eddies/Cyclonic/Cyclonic_2004*.nc'))
#cycl_files = sorted(glob.glob('/data/watkinson/Fall2022/Eddy_data/GoM_tuned_eddies/Cyclonic/Cyclonic_2004*.nc'))
#print(len(cycl_files))
'''
cycl_chlor_max = list()#empty list to load cyclonic eddy chlorophyll mean vlaues into throghout loop below
anti_chlor_max = list()#empty list to load anticyclonic eddy chlorophyll mean values into throghout loop below

cycl_chlor_min = list()#empty list to load cyclonic eddy chlorophyll mean vlaues into throghout loop below
anti_chlor_min = list()

n=0 #set n to 0 so the loop starts with first file in list
while n < len(sat_files): #once finalized, put this inside a funciton so you can calcualte the mean for multiple variables
    #Load in data
    a = EddiesObservations.load_file(anti_files[n])#anticyclonic eddy data
    c = EddiesObservations.load_file(cycl_files[n])#cyclonic eddy data
    sat_data = RegularGridDataset((sat_files[n]),"lon","lat",nan_masking=True)#satellite data
    
    #this interpolates the satellite data grid onto the contours of the eddy data and uses mean function to calcualte values inside these contours
        #can also do this method for: ‘center’, ‘mean’, ‘max’, ‘min’, ‘nearest’ 
    anti_chlor = a.interp_grid(sat_data, "chlor_a", method="mean", intern=False)#this interpolates the sa
    cycl_chlor = c.interp_grid(sat_data, "chlor_a", method="mean", intern=False)
    
    #anti_chlor_m = a.interp_grid(sat_data, "chlor_a", method="mean", intern=False)#this interpolates the sa
    #cycl_chlor_m = c.interp_grid(sat_data, "chlor_a", method="mean", intern=False)
    #append the result for each file loop to the variables below
    anti_chlor_max = np.concatenate([anti_chlor_max,anti_chlor])
    cycl_chlor_max = np.concatenate([cycl_chlor_max,cycl_chlor])
    
    #anti_chlor_min = np.concatenate([anti_chlor_min,anti_chlor_m])
    #cycl_chlor_min = np.concatenate([cycl_chlor_min,cycl_chlor_m])
    #anti_chlor_mean.append(anti_chlor)
    #cycl_chlor_mean.append(cycl_chlor)
    
    n=n+1


We assume pixel position of grid is centered for /data/watkinson/Fall2022/Sat_data/1day_regrid/CAFE_GOM_day_20030101.nc
We assume pixel position of grid is centered for /data/watkinson/Fall2022/Sat_data/1day_regrid/CAFE_GOM_day_20030102.nc
We assume pixel position of grid is centered for /data/watkinson/Fall2022/Sat_data/1day_regrid/CAFE_GOM_day_20030103.nc


1549
1549


In [69]:
print(anti_chlor_max)

[       nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan 0.38627264
        nan        nan        nan]
